## License Information

Copyright (C) 2024 Cyprien Quéméneur

FedPylot is released under the GPL-3.0 license, please refer to the LICENSE file in the root directory of the program.

For the full copyright notices, please refer to the NOTICE file in the root directory of the program.

## System Information

In [1]:
import sys
import torch

print(f"Python version: {sys.version}, {sys.version_info} ")
print(f"Pytorch version: {torch.__version__} ")
!nvidia-smi -L

Python version: 3.10.12 (main, Nov 20 2023, 15:14:05) [GCC 11.4.0], sys.version_info(major=3, minor=10, micro=12, releaselevel='final', serial=0) 
Pytorch version: 2.2.1+cu121 
GPU 0: Tesla V100-SXM2-16GB (UUID: GPU-5424c418-09d4-348a-6b02-27736bd997cc)


## Preparation

In [2]:
# for now the code, fully trained weights, and data are imported from a drive
from google.colab import drive

drive.mount('/content/gdrive')
!cp -r gdrive/MyDrive/fedpylot ./
drive.flush_and_unmount()
%cd fedpylot

Mounted at /content/gdrive
/content/fedpylot


## Benchmarking

In [3]:
import os
from google.colab import output
import pandas as pd

In [4]:
def run(data, bsz, trials, weights, spath, img, conf, iou):
    """Evaluate YOLOv7(-X) several times and print the mean inference speed."""
    if not os.path.exists(saving_path):
        os.makedirs(saving_path)
    for trial in range(trials):
        output.clear()  # only display the current trial
        !python yolov7/test.py --data $data --batch $bsz --kround $trial --weights $weights --saving-path $spath --project $spath --img $img --conf $conf --iou $iou --device 0
    results = pd.read_csv(os.path.join(saving_path, 'results.csv'))
    print('\n--------------------------------------------------------\n')
    if bsz == 1:
        t = round(1000 / results['batch avg time (ms)'].mean())
        print(f'Mean inference speed ({trials} trials, NMS excluded): {t} FPS')
    else:
        t = round(results['batch avg time (ms)'].mean(), 1)
        print(f'Mean inference speed ({trials} trials, NMS excluded): {t} ms')

### KITTI

In [5]:
# untar KITTI validation set
!tar -xf datasets/kitti/server.tar -C datasets/kitti

In [6]:
# parameters
data = 'data/kitti.yaml'       # *.data path
bsz = 1                        # batch size
trials = 20                    # number of inferences to run
conf_thres = 0.001             # object confidence threshold
iou_thres = 0.65               # IOU threshold for NMS

In [7]:
# YOLOv7-tiny
weights = 'weights/trained/kitti-yolov7-tiny-fedoptm.pt'
saving_path = 'datasets/kitti/yolov7-tiny-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/kitti/yolov7-tiny-bsz1', kround=19, weights=['weights/trained/kitti-yolov7-tiny-fedoptm.pt'], data='data/kitti.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/kitti/yolov7-tiny-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 200 layers, 6025525 parameters, 6025525 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor.

In [8]:
# YOLOv7
weights = 'weights/trained/kitti-yolov7-fedoptm.pt'
saving_path = 'datasets/kitti/yolov7-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/kitti/yolov7-bsz1', kround=19, weights=['weights/trained/kitti-yolov7-fedoptm.pt'], data='data/kitti.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/kitti/yolov7-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36517621 parameters, 36517621 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated 

In [9]:
# YOLOv7-X
weights = 'weights/trained/kitti-yolov7x-fedoptm.pt'
saving_path = 'datasets/kitti/yolov7x-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/kitti/yolov7x-bsz1', kround=19, weights=['weights/trained/kitti-yolov7x-fedoptm.pt'], data='data/kitti.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/kitti/yolov7x-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 354 layers, 70827253 parameters, 70827253 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you ac

In [10]:
# YOLOv7-W6
weights = 'weights/trained/kitti-yolov7-w6-fedoptm.pt'
saving_path = 'datasets/kitti/yolov7-w6-bsz1'
img_size = 1280

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/kitti/yolov7-w6-bsz1', kround=19, weights=['weights/trained/kitti-yolov7-w6-fedoptm.pt'], data='data/kitti.yaml', batch_size=1, img_size=1280, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/kitti/yolov7-w6-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 343 layers, 69840476 parameters, 69840476 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If

### nuImages-10

In [11]:
# untar nuImages-10 validation set
!tar -xf datasets/nuimages10/server.tar -C datasets/nuimages10

In [12]:
# parameters
data = 'data/nuimages10.yaml'  # *.data path
bsz = 1                        # batch size
trials = 5                     # number of inferences to run
conf_thres = 0.001             # object confidence threshold
iou_thres = 0.65               # IOU threshold for NMS

In [13]:
# YOLOv7-tiny
weights = 'weights/trained/nuimages10-yolov7-tiny-fedoptm.pt'
saving_path = 'datasets/nuimages10/yolov7-tiny-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages10/yolov7-tiny-bsz1', kround=4, weights=['weights/trained/nuimages10-yolov7-tiny-fedoptm.pt'], data='data/nuimages10.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages10/yolov7-tiny-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 200 layers, 6030919 parameters, 6030919 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on t

In [14]:
# YOLOv7
weights = 'weights/trained/nuimages10-yolov7-fedoptm.pt'
saving_path = 'datasets/nuimages10/yolov7-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages10/yolov7-bsz1', kround=4, weights=['weights/trained/nuimages10-yolov7-fedoptm.pt'], data='data/nuimages10.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages10/yolov7-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36528391 parameters, 36528391 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad fie

In [15]:
# YOLOv7-X
weights = 'weights/trained/nuimages10-yolov7x-fedoptm.pt'
saving_path = 'datasets/nuimages10/yolov7x-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages10/yolov7x-bsz1', kround=4, weights=['weights/trained/nuimages10-yolov7x-fedoptm.pt'], data='data/nuimages10.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages10/yolov7x-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 354 layers, 70840711 parameters, 70840711 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-lea

In [16]:
# YOLOv7-W6
weights = 'weights/trained/nuimages10-yolov7-w6-fedoptm.pt'
saving_path = 'datasets/nuimages10/yolov7-w6-bsz1'
img_size = 1280

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages10/yolov7-w6-bsz1', kround=4, weights=['weights/trained/nuimages10-yolov7-w6-fedoptm.pt'], data='data/nuimages10.yaml', batch_size=1, img_size=1280, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages10/yolov7-w6-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 343 layers, 69855860 parameters, 69855860 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the 

### nuImages-23

In [17]:
# untar nuImages-23 validation set
!tar -xf datasets/nuimages23/server.tar -C datasets/nuimages23

In [18]:
# parameters
data = 'data/nuimages23.yaml'  # *.data path
bsz = 1                        # batch size
trials = 5                     # number of inferences to run
conf_thres = 0.001             # object confidence threshold
iou_thres = 0.65               # IOU threshold for NMS

In [19]:
# YOLOv7-tiny
weights = 'weights/trained/nuimages23-yolov7-tiny-fedoptm.pt'
saving_path = 'datasets/nuimages23/yolov7-tiny-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages23/yolov7-tiny-bsz1', kround=4, weights=['weights/trained/nuimages23-yolov7-tiny-fedoptm.pt'], data='data/nuimages23.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages23/yolov7-tiny-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 200 layers, 6065980 parameters, 6065980 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on t

In [20]:
# YOLOv7
weights = 'weights/trained/nuimages23-yolov7-fedoptm.pt'
saving_path = 'datasets/nuimages23/yolov7-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages23/yolov7-bsz1', kround=4, weights=['weights/trained/nuimages23-yolov7-fedoptm.pt'], data='data/nuimages23.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages23/yolov7-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
RepConv.fuse_repvgg_block
Model Summary: 306 layers, 36598396 parameters, 36598396 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad fie

In [21]:
# YOLOv7-X
weights = 'weights/trained/nuimages23-yolov7x-fedoptm.pt'
saving_path = 'datasets/nuimages23/yolov7x-bsz1'
img_size = 640

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages23/yolov7x-bsz1', kround=4, weights=['weights/trained/nuimages23-yolov7x-fedoptm.pt'], data='data/nuimages23.yaml', batch_size=1, img_size=640, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages23/yolov7x-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 354 layers, 70928188 parameters, 70928188 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-lea

In [22]:
# YOLOv7-W6
weights = 'weights/trained/nuimages23-yolov7-w6-fedoptm.pt'
saving_path = 'datasets/nuimages23/yolov7-w6-bsz1'
img_size = 1280

run(data, bsz, trials, weights, saving_path, img_size, conf_thres, iou_thres)

Namespace(saving_path='datasets/nuimages23/yolov7-w6-bsz1', kround=4, weights=['weights/trained/nuimages23-yolov7-w6-fedoptm.pt'], data='data/nuimages23.yaml', batch_size=1, img_size=1280, conf_thres=0.001, iou_thres=0.65, task='val', device='0', single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project='datasets/nuimages23/yolov7-w6-bsz1', name='exp', exist_ok=False, no_trace=False, v5_metric=False)
YOLOR 🚀 2024-3-20 torch 2.2.1+cu121 CUDA:0 (Tesla V100-SXM2-16GB, 16151.125MB)

Fusing layers... 
Model Summary: 343 layers, 69955856 parameters, 69955856 gradients
 Convert model to Traced-model... 
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/module.py:836: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the 